In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm #視覺化進度用


In [2]:
#  ToSec 1 
def time_to_sec(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

In [3]:
#  ToSec 2
def convert_time_to_seconds(time):
    (h, m, s) = time.split(':')
    seconds = int(h) * 3600 + int(m) * 60 + int(s)
    return seconds

In [4]:
#  without value
def check_yield(df):
    condition = df['使用人時1'] == '無預計產量，無法計算'
    df.loc[df[condition].index, 'exception_message'] = 'No Yield'

In [20]:

# load excel
main_excel_path = '12_merged_2.xlsx'
main_df = pd.read_excel(main_excel_path, header=0)

In [21]:
# product_code_data = main_df.groupby('產品品號')

In [22]:
#  cheak product name 
main_df['check_name'] = main_df['製程代號'] + '_' + main_df['產品品號']
main_df['exception_message'] = ''

check_yield(main_df)

In [ ]:
#  data processing 
target_df = main_df[main_df['使用人時1'] != '無預計產量，無法計算']
#  time threshold that can change 
time_threshold = 30

i_check_name = 'A022_8MT007-C0151-71650-03'

check_list = np.unique(target_df['check_name'])
for i_check_name in tqdm(check_list):
    temp_df = target_df[target_df['check_name'] == i_check_name]
    temp_df_copy = temp_df.copy()
    temp_df_copy['prodcut_time'] = 0
    temp_df_copy['prodcut_time'] = temp_df_copy['使用人時1'].apply(time_to_sec)
    
    for df_idx in temp_df_copy.index:
              
        temp_time_diff = temp_df_copy.loc[:, 'prodcut_time'] - temp_df_copy.loc[df_idx, 'prodcut_time']
        for time_idx, i_time in enumerate(temp_time_diff):
            if np.abs(i_time) == 0:
                continue
            if np.abs(i_time) >= time_threshold:
                main_df_idx = temp_df_copy.index[time_idx]
                main_df.loc[df_idx, 'exception_message'] += temp_df_copy.loc[main_df_idx, '單號'] + ', '

 68%|██████▊   | 13669/20047 [03:07<01:14, 85.71it/s]

In [ ]:
#  output
final_df = main_df[main_df['exception_message'] != '']
final_df2 = main_df[main_df['exception_message'] == '']

final_df.to_excel('12_checked_worng.xlsx', index = False)
final_df2.to_excel('12_checked_correct.xlsx', index = False)